<a href="https://colab.research.google.com/github/edoardochiarotti/class_datascience/blob/main/2023/05_Mean-Model/Resources/05_Mean_model_exercises_solutions.ipynb" target="_blank" rel="noopener"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Mean Model - Exercises with Solutions

<img src="https://i.imgflip.com/82uqjd.jpg" width="500">

In [1]:
# PACKAGES
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statistics as st
import statsmodels.api as sm

# FUNCTIONS FROM PACKAGES
from numpy.linalg import inv

# SEABORN THEME
scale = 0.4
W = 16*scale
H = 9*scale
sns.set(rc = {'figure.figsize':(W,H)})
sns.set_style("white")

Main references:
- For the part on the functions for the sample mean and variance, we took inspiration from the amazing class on [Multiple Equation Estimation](https://www.fionaburlig.com/teaching/are212) by Fiona Burlig (with R) at UC Berkeley.
- For a refresher on matrixes and matrixes in Python, see the [Notebook on matrixes](https://github.com/edoardochiarotti/class_datascience/blob/main/2023/05_Mean-Model/Resources/05_Matrixes.ipynb).

## Content
- [Exercise 1: Load data](#Exercise-1:-Load-data)
- [Exercise 2: Debugging the Mean Estimator](#Exercise-2:-Debugging-the-Mean-Estimator)
- [Exercise 3: Write Nice Functions for your Estimators](#Exercise-3:-Write-Nice-Functions-for-your-Estimators)
- [Exercise 4: Statistical Test with the Sample Mean](#Exercise-4:-Statistical-Test-with-the-Sample-Mean)

## Exercise 1: Load data <a name="Exercise-1:-Load-data"></a>
- Let's now use **Python** to write down the functions for the sample-mean and sample-variance estimators using matrix notation.
- We'll test these functions on data from the **QoG Environmental Indicators dataset (QoG-EI)** (Povitkina, Marina, Natalia Alvarado Pachon & Cem Mert Dalli. 2021). The Quality of Government Environmental Indicators Dataset, version Sep21 (University of Gothenburg: [The Quality of Government Institute](https://www.gu.se/en/quality-government)), is a compilation of indicators measuring countries' environmental performance over time, including the presence and stringency of environmental policies, environmental outcomes (emissions, deforestation, etc.), and public opinion on the environment. Codebook and data are available [here](https://www.gu.se/en/quality-government/qog-data/data-downloads/environmental-indicators-dataset).
- Note that we could test theste functions also on generated data, but it's nicer to test them on real-life data.
- Get the QoG dataset using this link `https://www.qogdata.pol.gu.se/data/qog_ei_sept21.xlsx`

In [2]:
# your code here ...

# get data
link = "https://www.qogdata.pol.gu.se/data/qog_ei_sept21.xlsx"
df_qog = pd.read_excel(link)

- For this exercise we'll use the sample mean to estimate the mean of **CO2 emissions per capita**. In QoG, the variable for total CO2 emissions per capita is `edgar_co2pc`.
- Create a subset of the dataset you have downloaded with only the variables `ccodealp`, `year` and `edgar_co2pc`, and display it. 

In [8]:
# your code here ...

# indexes
indexes = ["ccodealp","year"]

# CO2 variables
variabs_co2 = ["edgar_co2pc"]

# subset
df_qog_sub = df_qog.loc[:,np.append(indexes,variabs_co2)]
df_qog_sub

,ccodealp,year,edgar_co2pc
0,AFG,1946,NaN
1,AFG,1947,NaN
2,AFG,1948,NaN
3,AFG,1949,NaN
4,AFG,1950,NaN
...,...,...,...
11717,VDR,1972,NaN
11718,VDR,1973,NaN
11719,VDR,1974,NaN
11720,VDR,1975,NaN


- As this is a panel dataset and we want to start simple
    - Collapse the subset database into **cross section** of countries by taking the mean of all variables by country. When you do this, keep the variable `ccodealp` as a variable, and not as an index of the dataframe.
    - Drop all missing values, as our sample mean estimator will work only on a clean dataset.

In [7]:
# your code here ...

# make cross section
df = df_qog_sub.groupby("ccodealp")[variabs_co2].mean().reset_index().dropna()
df

,ccodealp,edgar_co2pc
0,AFG,0.161829
1,AGO,1.093760
2,ALB,1.800099
3,ARE,35.023444
4,ARG,3.965821
...,...,...
173,WSM,0.631124
174,YEM,0.804577
175,ZAF,8.327993
176,ZMB,0.423897


- You should have obtained a dataframe with 178 rows, i.e., 178 countries.
- Now we want to estimate the mean of our variable of interest emissions per capita `edgar_co2pc`. Let's do the **usual reasoning** shall we? Our unit for this analysis are the countries. There is a population of countries in the world with their emissions per capita, represented by a normal probability distribution of emissions with mean $\beta$ and variance $\sigma^2$ . We cannot work with the population, but we can work only with a sample of emissions for 178 countries we have at hand (which is the realization of a random sample). This sample is just a representation of the population of all countries in the world, which amounts to 196 countries (yeah, in this case our sample is very close to the full population, but still it's not the full population).
- The sample-mean estimator is a good estimator for the average emission $\beta$. As mentioned, if we define a N-dimensional vector of 1s $\boldsymbol{x}=(1,...,1)'$, the sample-mean estimator can be expressed as $\hat{\beta}= (\boldsymbol{x}'\boldsymbol{x})^{-1}(\boldsymbol{x}'\boldsymbol{y})$. 
- So let's add this **vector of ones** to our dataset. Consider the collapsed dataframe you have obtained in the former question and add a column of ones, and display the data.

In [ ]:
# your code here ...

# put ones into data
df["ones"] = 1
df

## Exercise 2: Debugging the Mean Estimator <a name="Exercise-2:-Debugging-the-Mean-Estimator"></a>
- We can now write our **function for the sample mean estimator**. As we need to work with matrixes, the first thing to do is to convert panda data into numpy arrays (what Python uses for matrixes). Once the conversion is done, we can write the formula of our estimator by using the functions for matrix operations (multiplication, transpose, inverse) that we have learned in Python.
- Let's give it a first try with a function that takes as arguments the dataframe, the name of our variable of interest (y) and the name of our single regressor, which is the vector of ones (x).
- I report here below an example of this function. By reading the lines you should be able to understand what this does.

In [ ]:
# define sample mean function
def sample_mean_estimator(data, y, x):
    
    """ My Sample Mean Function """
    
    # store in matrixes
    ydata = data.loc[:,y].to_numpy()
    xdata = data.loc[:,x].to_numpy()

    # get sample mean
    beta_hat = (inv(xdata.T @ xdata)) @ (xdata.T @ ydata)

    # return
    return float(beta_hat)

- Use this function to obtain the sample mean of the variable `edgar_co2pc` (remember you also have the argument `x` to fill in ...).

In [ ]:
# your code here ...

# try function
sample_mean_estimator(data = df, y = "edgar_co2pc", x = "ones")

- The function should give you the error "LinAlgError", so there must be something wrong with our linear algebra? In addition, the error says `0-dimensional array given. Array must be at least two-dimensional`. Possibly we are feeding a function with a 0-dimensional array, though this function wants an array that is at least two-dimensional. 
- Debug the `sample_mean_estimator(data, y, x)` function, i.e. find the bug (find the line of code that gives the error). Tips:
    - Use as many cells as you need (here below I have just put one, you can add more)
    - As first step, you should store the arguments you have used for the function as objects in the general environment in which you are working. Translated, that means store `df` as `data`, `"edgar_co2pc"` as `y` and `"ones"` as `x`. Then, copy the body of your function and re-run.
    - The bug has something to do with how Python applies certain operations to scalars.

In [ ]:
# your code here ...

# debug
data = df
y = "edgar_co2pc"
x = "ones"

# store in matrixes
ydata = data.loc[:,y].to_numpy()
xdata = data.loc[:,x].to_numpy()

# get sample mean
beta_hat = (inv(xdata.T @ xdata)) @ (xdata.T @ ydata)

In [ ]:
xdata.T @ xdata

In [ ]:
inv(xdata.T @ xdata)

- Find a solution for this bug and update the function `sample_mean_estimator` with this solution. Tips:
    - There are more solutions, for example one has to do with the syntax `[[]]`, and another one has to do with the function `np.atleast_2d`. 
    - The second one is the more general one and the one to focus on.
    - The update should be a chunk of code between the chunks `# store in matrixes` and `# get sample mean`.
    - Feel free to use as many cells as you need.
- Run the function and compute the sample average of CO2 emissions per capita. How much is it?

In [ ]:
# your code here ...
solution = np.array([[xdata.T @ xdata]])
print(solution.shape)
print(inv(solution))

In [ ]:
# define better sample mean function
def sample_mean_estimator(data, y, x):
    
    """ My Sample Mean Function """
    
    # store in matrixes
    ydata = data.loc[:,y].to_numpy()
    xdata = data.loc[:,x].to_numpy()
    
    # make column vectors for arrays with less than 2 dimensions
    if len(ydata.shape) == 1:
        ydata = np.atleast_2d(ydata).T
    if len(xdata.shape) == 1:
        xdata = np.atleast_2d(xdata).T

    # get sample mean
    beta_hat = (inv(xdata.T @ xdata)) @ (xdata.T @ ydata)

    # return
    return float(beta_hat)

In [ ]:
# run the function
sample_mean_estimator(data = df, y = "edgar_co2pc", x = "ones")

## Exercise 3: Write Nice Functions for your Estimators <a name="Exercise-3:-Write-Nice-Functions-for-your-Estimators"></a>
- Now we have one function for the sample mean estimator (with the fix). True Python coders would tell you that an efficient function should not have more than 5 lines max. So to have a **clean and efficient code**, what you should do is split this single function in multiple functions. 
- There are many ways to do this, but let's follow the logic of the function at hand:
    - The first chunk of your function `sample_mean_estimator(data, y, x)` reads data and transforms it into matrixes. Take that chunk out and write a stand-alone function that transforms data into matrixes, which you can call `data_to_matrix(data, variab_name)` (your function has 2 arguments).
    - Tips: you can also look at the code of the updated function `sample_mean_estimator(data, y, x)` below as a hint on what you are supposed to do here.

In [ ]:
# your code here ...

# function to transform panda series into vectors / matrices
def data_to_matrix(data, variab_name):
    
    """ My Data to Matrix Function """
    
    # store in matrixes
    matrix = data.loc[:,variab_name].to_numpy()
    
    # make column vectors for arrays with less than 2 dimensions
    if len(matrix.shape) == 1:
        matrix = np.atleast_2d(matrix).T
        
    # return result
    return matrix

- Now your function `sample_mean_estimator(data, y, x)` only has the second chunk of code. We should re-define it so that it uses the function `data_to_matrix`. I give you the solution here:

In [ ]:
# define sample mean function
def sample_mean_estimator(data, y, x):
    
    """ My Sample Mean Function """
    
    # store in matrixes
    ydata = data_to_matrix(data, variab_name = y)
    xdata = data_to_matrix(data, variab_name = x)

    # get sample mean
    beta_hat = (inv(xdata.T @ xdata)) @ (xdata.T @ ydata)

    # return
    return float(beta_hat)

- Run this updated function to get the sample mean estimate of CO2 emissions per capita, and compare the results with a built in ("canned") function called `st.mean` using a logical operator (such as `np.allclose`). Tips:
    - The formulas inside the function `st.mean` can be a bit different than our matrix formulas (you should look inside by using `??`). For this reason, the output might come out some decimals different, so the operator `==` will not work.

In [ ]:
# your code here ...

# estimate sample mean and store
sample_mean = sample_mean_estimator(data = df, y = "edgar_co2pc", x = "ones")
sample_mean

# use canned method and compare
sample_mean_canned = st.mean(df.loc[:,"edgar_co2pc"])
print(sample_mean_canned)
np.allclose(sample_mean, sample_mean_canned)

- Write a function for the sample variance estimator and obtain the sample variance estimate for CO2 emissions per capita. Tips:
    - you should call this function `sample_variance_estimator(data, y, x, sample_mean)` (it has 4 arguments, where the 4th one is the outcome of your sample-mean function). 
    - This function should have a similar structure than the function `sample_mean_estimator`, for example, the first part should use the function `data_to_matrix`, and the second part should apply the matrix formulas of the sample-variance estimator.
    - As an intermediate step to get the sample variance, you can get the residuals like this `resid = (ydata - np.dot(sample_mean, xdata))`, and then use them to get the sample variance (all from the matrix formulas).

In [ ]:
# your code here ...

# write function for estimator for the standard error of the mean
def sample_variance_estimator(data, y, x, sample_mean):

    """ My Sample Variance Function """
    
    # store in matrixes
    ydata = data_to_matrix(data, variab_name = y)
    xdata = data_to_matrix(data, variab_name = x)

    # get params
    N = len(ydata)
    degrees_freedom = N-1

    # get sample variance
    resid = (ydata - np.dot(sample_mean, xdata))
    sigma2_hat = (resid.T @ resid) /degrees_freedom
    
    # return
    return float(sigma2_hat)

In [ ]:
sample_variance = sample_variance_estimator(data = df, y = "edgar_co2pc", x = "ones", 
                                            sample_mean = sample_mean)
sample_variance

- Write a stand-alone function for the estimator Standard Error of the Mean (SEM) and obtain the SEM estimate for CO2 emissions per capita. Tips:
    - you should call this function `sem_estimator(data, y, x, sample_mean)` (it has 4 arguments, where the 4th one is the outcome of your sample-mean function). 
    - It should have a similar structure than the function `sample_variance_estimator`

In [ ]:
# your code here ...

# write function for estimator for the standard error of the mean
def sem_estimator(data, y, x, sample_mean):

    """ My SEM Estimator Function """
    
    # store in matrixes
    ydata = data_to_matrix(data, variab_name = y)
    xdata = data_to_matrix(data, variab_name = X)

    # get params
    N = len(ydata)
    degrees_freedom = N-1

    # get sample variance
    resid = (ydata - np.dot(sample_mean, xdata))
    sigma2_hat = (resid.T @ resid) /degrees_freedom
    
    # get sem
    sigma_hat = np.sqrt(sigma2_hat)
    sem_hat = sigma_hat / np.sqrt(N)
    
    # return
    return float(sem_hat)

In [ ]:
sem = sem_estimator(data = df, y = "edgar_co2pc", x = "ones", sample_mean = sample_mean)
sem

## Exercise 4: Statistical Test with the Sample Mean <a name="Exercise-4:-Statistical-Test-with-the-Sample-Mean"></a>


- Let's now rely on the knowledge we have on **statistical tests** from the exercises of last class to write a function that does a one sample z-test on whether the estimate of the mean is significantly different from zero at the 5% level.
- Remember? In the exercises, we have done (i) z-test with known variance, with z-score following a standard normal distribution, (ii) t-test with unknown variance, with t-statistic following a T-student distribution with $N-1$ degrees of freedom. 
- Thanks to the Central Limit Theorem (which we saw last time), we know that, as the sample size increases, the sample-mean estimator $\hat{\beta}$ always follows a normal distribution. It turns out that, thanks to the same theorem, as the sample size increases, the t-statistic also follows a standard normal distribution. Therefore, we can do a test with unknown variance using the functions and critical values of the normal distribution, such as `st.NormalDist()` and `stats.norm.ppf()`, rather than the functions and critical values of the t-student distribution (i.e. `stats.t.cdf()` and `stats.t.ppf()`).
- Write a function that computes the z-score, p value, and the confidence interval for the sample mean and use it to test if the population mean of CO2 emissions per capita is different than zero, considering the estimate of the sample mean obtained above. Tips:
    - Most of the code is in the exercises of last class
    - You can call your function `one_sample_z_test(sample_mean, sem, beta_null = 0, significance_level = 0.05)` (4 arguments, with the first two being the obtained estimate for the sample mean and the obtained estimate for the standard error of the mean).


In [ ]:
# write function for one sample t test statistic
def one_sample_z_test(sample_mean, sem, beta_null = 0, significance_level = 0.05):
    
    """ My Z Test Function """

    # get z score
    z_score = (sample_mean - beta_null) / sem

    # get p value
    lower_area = st.NormalDist().cdf(-abs(z_score))
    upper_area = lower_area
    p_value = lower_area + upper_area

    # get confidence interval
    alpha_inv = (1.0-significance_level)
    q1 = (1+alpha_inv)/2
    ci_critical = stats.norm.ppf(q1)
    ci = (sample_mean-(ci_critical*sem), sample_mean+(ci_critical*sem))

    # return results
    return (z_score, p_value, ci)

- Use the function to test if the mean of the population of CO2 emissions per capita is different from zero and compare the results with a built-in ("canned") function `stats.weightstats.ztest` from the `statsmodels` package, using a logical operator. Tips:
    - Again you should be using `np.allclose` and not `==`.

In [ ]:
# your code here ...

# run and store z test
z_test = one_sample_z_test(sample_mean = sample_mean, sem = sem)
z_test

In [ ]:
# compare with canned methods
import statsmodels
z_test_canned = statsmodels.stats.weightstats.ztest(df["edgar_co2pc"])
print(z_test[0:2])
print(z_test_canned)
print(np.allclose(z_test[0:2], z_test_canned))

z_test_ci_canned = statsmodels.stats.weightstats.zconfint(x1 = df["edgar_co2pc"], x2=None, value=0, alpha=0.05)
print(z_test[2])
print(z_test_ci_canned)
print(np.allclose(z_test[2], z_test_ci_canned))

- OK so to **recap** you have written functions for sample-mean, variance and standar-error estimators, and you have obtained estimates from all of these on a sample of CO2 emissions per capita at the country level. You have also run a statistical test to test whether the sample-mean estimate is significantly different than 0 at the 5% confidence level, using both t statistic / p value and confidence interval. 
- Use the functions developed above to display the results all together here below and comment them using the statistical "phrasing" you have learned. Tips:
    - This should not take you more than 5 lines of code.
    - The phrasing should be something along the lines of "The sample mean for CO2 emissions per capita is ... and based on this estimate and our statistical test we can tell that the mean of the population is ...".

In [ ]:
# your code here

# estimate sample mean and store
sample_mean = sample_mean_estimator(data = df, y = "edgar_co2pc", x = "ones")

# get p value of estimate being different than 0
sem = sem_estimator(data = df, y = "edgar_co2pc", x = "ones", sample_mean = sample_mean)
p_value = one_sample_z_test(sample_mean = sample_mean, sem = sem, beta_null = 0)[1]

# print
print("Estimate of Beta from Sample Mean Estimator:          ", round(sample_mean, 3))
print("P-value for Estimate of Beta being different than 0:  ", round(p_value, 3))

Your answer here ...

The coefficient 5.016 tells us that, on average, one person emits 5.016 of CO2 emissions per year. In addition, looking at the p-value of 0.000, we can say that this result is statistically significant at the 1% level (the p-value is smaller than 0.01). This means that, based on our sample and assumptions, we can be pretty darn sure that a person emits more than 0 tons of CO2 per year. The probability that we are wrong on this claim is 0.00000000000038846703634227 %.